In [1]:
##check for gpu
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1c2d5837-45db-14c5-090f-013573e3de8b)


##Get helper Functions



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import serires of funtions 
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2021-08-11 02:34:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-11 02:34:13 (109 MB/s) - ‘helper_functions.py’ saved [10246/10246]



##Get a text dataset

link :https://www.kaggle.com/c/nlp-getting-started


In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-11 02:34:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-08-11 02:34:15 (147 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##Visulaizing the dataset

In [4]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#shuffle training dataframe
train_df_shuffled =train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
#test dataframe look
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#count the examples
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(test_df),len(train_df)

(3263, 7613)

In [9]:
#visulalize random samples
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
   _,text,target=row
   print(f"Target :{target}","(real disater)" if target > 0 else "(not real disater)")
   print(f"Text :\n {text}\n")
   print("---\n")

Target :0 (not real disater)
Text :
 A tornado flew around my room before you came

---

Target :0 (not real disater)
Text :
 New Nanotech Device Will Be Able To Target And Destroy Blood Clots http://t.co/HFy5V3sLBB

---

Target :0 (not real disater)
Text :
 Floods Fishing Finally Sunshine &amp; Fab Deals from Albertsons Anniversary Sale |Lauren Paints | a beautiful life http://t.co/CwHSLMB8x9

---

Target :0 (not real disater)
Text :
 the windstorm blew thru my open window and now my bong is in pieces just another example of nature's indifference to human suffering

---

Target :1 (real disater)
Text :
 Cont'd- #Sinjar: referring to a 40-pg document the group put together on the movt of #Yazidi hostages in the days following #IS massacre [2]

---



##Split data into train and Validation

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
#use train test split 
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [12]:
#check the length
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

##Converting Text into numbers

when dealing with text problem ,

1. builld the model to convert the words into numbers
  
  *Tokanization
  
  *embedding



##Tokenization

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [15]:
#use the defult text vectorization parameters
text_vectorizer =TextVectorization(max_tokens=None,#how many words taken into token
                                   standardize="lower_and_strip_punctuation",
                                   split="whitespace",
                                   ngrams=None,
                                   output_mode="int",
                                   output_sequence_length=None,
                                   pad_to_max_tokens=True)

In [16]:
text_vectorizer

In [17]:
#find the avarge number of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
#setup the text vectorization varibales
max_vocab_length=10000 #max number in vocabulary
max_length=15 #max length our sequences 

text_vectorizer =TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)



In [19]:
#fit the text vectorizer to train data
text_vectorizer.adapt(train_sentences)

In [20]:
#create a sample sentence and tokenize it
sample_sentence ="there is fire in my town"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,  42,   4,  13, 801,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
#choose random sentence from the training daatset
random_sentence=random.choice(train_sentences)
print(f"Orignal text :\n {random_sentence} \
      \n\nVectorized version:")

text_vectorizer([random_sentence])

Orignal text :
 'We Can Help' Says Denver Firefighter Working To Curb First Responder Suicide http://t.co/aVV6hPNpch       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  46,   71,  148,  296, 1145, 1674, 1049,    5, 3952,   97, 4748,
          87,    1,    0,    0]])>

In [22]:
#get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() #get all the unique words in train data
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[:-5]

print(f"Number of words in vocab :{len(words_in_vocab)}")
print(f"5 most common words in vocab :{top_5_words}")
print(f"5 least common of words in vocab :{bottom_5_words}")

Number of words in vocab :10000
5 most common words in vocab :['', '[UNK]', 'the', 'a', 'in']
5 least common of words in vocab :['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'yo

##Create a Embedding using Embedding Layer

PARAMETERS:

1.INPUT DIM-size of the vocabulary

2.OUTPUT DIM -size of the output embedding vector

3.INPUT LENGTH-length of the sequence being passed to the embedding layer

In [23]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

embedding

In [24]:
random_sentence=random.choice(train_sentences)
print(f"Orignal Text :\n{random_sentence}\
      \n\nEmbedded Vesrion :")

#embed the setence
sample_embed =embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal Text :
[Latest Post] Bayelsa poll: Tension in Bayelsa as Patience Jonathan plans to hijack APC PDP http://t.co/B2yvLMPepR      

Embedded Vesrion :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02303665,  0.0114466 , -0.01523482, ..., -0.01601164,
         -0.0497044 ,  0.01257416],
        [-0.04292778,  0.03439453,  0.02118016, ...,  0.00562521,
          0.01051906, -0.03772955],
        [ 0.02336459,  0.00405455,  0.00086869, ...,  0.03852114,
          0.00025998,  0.01426954],
        ...,
        [ 0.02304903, -0.02761325,  0.01721989, ...,  0.04525675,
          0.02398804,  0.03806025],
        [-0.04917971, -0.00799701,  0.00097314, ...,  0.03753675,
         -0.04385232,  0.01782247],
        [ 0.00038626,  0.01685429, -0.00985099, ..., -0.02227091,
          0.02925951, -0.00156194]]], dtype=float32)>

In [25]:
#check out single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.30366476e-02,  1.14465952e-02, -1.52348168e-02, -8.80380720e-03,
         1.07149370e-02,  4.71296795e-02,  4.97081131e-03, -3.86573896e-02,
         3.94860767e-02, -3.15460451e-02,  3.64146717e-02,  9.16001946e-03,
         2.18942650e-02, -4.29932736e-02, -3.54506820e-03,  4.05391194e-02,
         3.14088501e-02,  3.76413949e-02, -4.44047451e-02, -7.75470585e-03,
        -1.69035792e-02, -2.72835977e-02, -9.66336578e-03,  3.21725868e-02,
        -6.99242204e-03, -4.16604653e-02, -3.89593840e-03, -4.21784297e-02,
        -8.45182687e-04, -4.04836908e-02, -4.00381461e-02,  9.95001942e-03,
         3.71463932e-02,  8.32419470e-03, -1.78465955e-02, -1.70119405e-02,
        -1.40307918e-02,  2.99491324e-02,  9.71995667e-03, -4.14220206e-02,
        -6.53974712e-05, -3.84048931e-02, -4.27799299e-03,  2.81590223e-03,
        -1.05667934e-02,  5.55812195e-03,  4.49865125e-02, -2.60388609e-02,
        -2.33876240e-02, -3.97033207e-0

##Modeling a Text Dataset and Runnning Experiments

model 0-naive bayes(baseline)

model 1-Feed forward neaural Network(dense model)

model 2-LSTM Model

model 3-GRU Model

model 4-Bidirectional Model

model 5-ID Convolutional Neural Network Model

model 6-Tensorflow hub pre trained feature extractor Model

model 7-same model with 10% training data Model

*steps:

+create the model

+compile the model

+fit the model

+evaluate the model


##Model 0 -Getting a Baseline model

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokanization and modeling pipeline
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clf",MultinomialNB())
])

#fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
#evaluate the baseline model
baseline_score=model_0.score(val_sentences,val_labels)
print(f"model arcive score :{baseline_score*100:.2f}")
baseline_score

model arcive score :79.27


0.7926509186351706

In [28]:
#make predictions
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

##Create evaluation Function

Accuracy

precition

recall

f1-score


In [29]:
#funtion to evaluate
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calcualte_results(y_true,y_preds):
  """
  Calculates model accuracy,preciton,recall and f1 score of a binary classification model
  """

  #calcualte model accuracy
  model_accuracy=accuracy_score(y_true,y_preds)*100

  #calculate model prestion,recall,f1-score 
  model_precision,model_recall,model_f1,_ =precision_recall_fscore_support(y_true,y_preds,average="weighted")
  model_results={"accuracy":model_accuracy,
                 "precision":model_precision,
                 "recall":model_recall,
                 "f1":model_f1}

  return model_results



In [30]:
#get baseline results
baseline_results =calcualte_results(y_true=val_labels,
                                    y_preds=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 1 -Build a Dense Model

In [31]:
#crate a tensorboard callbacks
from helper_functions import create_tensorboard_callback

#create directory to save logs
SAVE_DIR="model_logs"

In [32]:
#build model with functional API
from tensorflow.keras import layers

inputs =layers.Input(shape=(1,),dtype =tf.string)

#turn text into numbers
x=text_vectorizer(inputs)

#create an embedding of the numberized inputs
x=embedding(x)

x=layers.GlobalAveragePooling1D()(x)

outputs=layers.Dense(1,activation="sigmoid")(x)

model_1=tf.keras.Model(inputs,outputs,name="model_1")

In [33]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
#fit the model
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])


Saving TensorBoard log files to: model_logs/model_1_dense/20210811-023423
Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.6127 - accuracy: 0.6906 - val_loss: 0.5385 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4419 - accuracy: 0.8193 - val_loss: 0.4691 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3462 - accuracy: 0.8608 - val_loss: 0.4620 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2842 - accuracy: 0.8921 - val_loss: 0.4601 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2369 - accuracy: 0.9107 - val_loss: 0.4832 - val_accuracy: 0.7900


In [36]:
#check the results
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4832 - accuracy: 0.7900


[0.4831780791282654, 0.7900262475013733]

In [37]:
#make predictions
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs

array([[3.25458497e-01],
       [7.59223104e-01],
       [9.97973859e-01],
       [1.12442151e-01],
       [8.78196061e-02],
       [9.38319266e-01],
       [9.23259258e-01],
       [9.94281173e-01],
       [9.65280831e-01],
       [2.75908619e-01],
       [1.14045396e-01],
       [6.39116168e-01],
       [4.22981307e-02],
       [2.78142929e-01],
       [3.89904273e-03],
       [1.08138055e-01],
       [2.66847666e-02],
       [5.73855899e-02],
       [2.44185045e-01],
       [4.66645330e-01],
       [8.66889119e-01],
       [3.87129933e-02],
       [4.05610591e-01],
       [6.78299889e-02],
       [9.55392480e-01],
       [9.98762727e-01],
       [3.51302698e-02],
       [5.63707575e-02],
       [2.20480766e-02],
       [1.97448313e-01],
       [5.11433125e-01],
       [2.56732553e-01],
       [4.22313631e-01],
       [1.67949006e-01],
       [4.51423496e-01],
       [5.57602048e-02],
       [9.94131386e-01],
       [1.36174768e-01],
       [3.03509533e-02],
       [9.98308539e-01],


In [38]:
#convert predicions to label format
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [39]:
#calculte model_1_results
model_1_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_1_preds)

model_1_results

{'accuracy': 79.00262467191601,
 'f1': 0.7863113264688066,
 'precision': 0.7975156487081627,
 'recall': 0.7900262467191601}

In [40]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Visializing Laerned Embeddings

In [41]:
#get the vocabulary from the text vectorization
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
#Model 1 summary
model_1.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [43]:
#get the weight metrics of embedding layer
embed_weights =model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [44]:
embed_weights

array([[-0.00077203, -0.05199467, -0.01041779, ..., -0.01021545,
        -0.04683018, -0.0202005 ],
       [-0.03339993, -0.03410576, -0.04451986, ...,  0.0484458 ,
         0.02604397,  0.01591387],
       [-0.06898703, -0.01702738,  0.02223215, ...,  0.04915821,
        -0.0683106 ,  0.00766769],
       ...,
       [ 0.01421429,  0.00441519, -0.00614349, ..., -0.03065876,
         0.02195785,  0.04862291],
       [-0.05850152, -0.05878484, -0.03068361, ...,  0.01254087,
        -0.05816957,  0.00934697],
       [-0.03667343, -0.09656888, -0.09263226, ...,  0.0156646 ,
        -0.10806537, -0.03498603]], dtype=float32)

In [45]:
#create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [46]:
#downlaod files from colab to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Model 2-Recurrent Neural Network

Structure of the RNN:

Input-->Tokenize-->Embedding-->Layers-->Output

In [50]:
#cerate an lstm model
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)

# x=layers.LSTM(units=64,return_sequences=True)(x)
# print(x.shape)

x=layers.LSTM(64)(x)
print(x.shape)

# x=layers.Dense(64,activation="relu")(x)
# print(x.shape)

outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [51]:
#get the summary
model_2.summary()


Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [52]:
#compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [55]:
#fit the model
model_2_history =model_2.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210811-030728
Epoch 1/5
215/215 [==============================] - 4s 19ms/step - loss: 0.1551 - accuracy: 0.9419 - val_loss: 0.6511 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1301 - accuracy: 0.9510 - val_loss: 0.6000 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1046 - accuracy: 0.9602 - val_loss: 0.8583 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0850 - accuracy: 0.9656 - val_loss: 0.9312 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0700 - accuracy: 0.9712 - val_loss: 0.9268 - val_accuracy: 0.7769


In [57]:
#make predictions 
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[2.1479419e-02],
       [7.8341895e-01],
       [9.9979454e-01],
       [5.8098383e-02],
       [3.4976122e-04],
       [9.9761093e-01],
       [7.1740723e-01],
       [9.9983644e-01],
       [9.9972695e-01],
       [4.5094138e-01]], dtype=float32)

In [58]:
#convert model 2 pred to labels
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [61]:
#calculate the model results
model_2_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_2_preds)

model_2_results

{'accuracy': 77.69028871391076,
 'f1': 0.7747522807746444,
 'precision': 0.7789562668302826,
 'recall': 0.7769028871391076}

##Model 3 -GRU -Gated Recurrent Unit



In [72]:
#build rnn using the gru unit
from tensorflow.keras import layers
inputs =layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)

#when stack recurrent layers ,return_sequences must be "True"
x=layers.GRU(64)(x)
# x=layers.LSTM(64,return_sequences=True)(x)
# x=layers.GRU(64,return_sequences=True)(x)
# x=layers.Dense(64,activation="relu")(x)
# x=layers.GlobalAveragePooling1D()(x)
outputs =layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")



In [69]:
#Model Summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_2 (GRU)                  (None, 15, 64)            37248     
_________________________________________________________________
dense_8 (Dense)              (None, 15, 1)             65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [73]:
model_3.summary()


Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_4 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [74]:

#compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [75]:
#fit the model
model_3_history =model_3.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20210811-035352
Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.1490 - accuracy: 0.9439 - val_loss: 0.6754 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0795 - accuracy: 0.9698 - val_loss: 0.9254 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0655 - accuracy: 0.9743 - val_loss: 0.9146 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0514 - accuracy: 0.9775 - val_loss: 1.2570 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0529 - accuracy: 0.9762 - val_loss: 1.1791 - val_accuracy: 0.7756


In [76]:
#make predictions 
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[7.3960429e-04],
       [6.4545459e-01],
       [9.9992752e-01],
       [1.1198292e-01],
       [6.9359274e-05],
       [9.9980730e-01],
       [6.0618526e-01],
       [9.9996257e-01],
       [9.9992144e-01],
       [9.4971013e-01]], dtype=float32)

In [77]:
#convert model 2 pred to labels
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [78]:
#calculate the model results
model_3_results=calcualte_results(y_true=val_labels,
                                y_preds=model_3_preds)
model_3_results


{'accuracy': 77.55905511811024,
 'f1': 0.7726614113025507,
 'precision': 0.7794121823201254,
 'recall': 0.7755905511811023}

In [79]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 4 -Bi directional RNN

In [88]:
#build the bidirectinal layer
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)

# x=layers.Bidirectional(layers.LSTM(32,return_sequences=True))(x)
# print(x.shape)

x=layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)

outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

(None, 15, 128)
(None, 128)


In [89]:
#model summary
model_4.summary()


Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [90]:
#compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [91]:
#fit the model
model_4_history =model_4.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "Bidirectional")])

Saving TensorBoard log files to: model_logs/Bidirectional/20210811-044747
Epoch 1/5
215/215 [==============================] - 8s 25ms/step - loss: 0.1030 - accuracy: 0.9710 - val_loss: 1.0746 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0497 - accuracy: 0.9788 - val_loss: 1.3166 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0451 - accuracy: 0.9800 - val_loss: 1.2471 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0390 - accuracy: 0.9818 - val_loss: 1.6857 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0382 - accuracy: 0.9819 - val_loss: 1.5007 - val_accuracy: 0.7743


In [92]:
#make predictions 
model_4_pred_probs=model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[8.6198905e-03],
       [6.7564130e-01],
       [9.9997652e-01],
       [1.7589431e-01],
       [6.4235451e-06],
       [9.9991775e-01],
       [8.1511474e-01],
       [9.9999034e-01],
       [9.9998164e-01],
       [9.9910885e-01]], dtype=float32)

In [93]:
#convert model 2 pred to labels
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [94]:
#calculate the model results
model_4_results=calcualte_results(y_true=val_labels,
                                y_preds=model_4_preds)
model_4_results


{'accuracy': 77.42782152230971,
 'f1': 0.7725974162749719,
 'precision': 0.7752857985262857,
 'recall': 0.7742782152230971}

In [95]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [96]:
model_3_results

{'accuracy': 77.55905511811024,
 'f1': 0.7726614113025507,
 'precision': 0.7794121823201254,
 'recall': 0.7755905511811023}

##Model 5-Convolutional Neural Networks for Text(Other types of sequence data)

In [100]:
#test out embedding ,conv1D and max pooling
from tensorflow.keras import layers
embedding_test=embedding(text_vectorizer(["this is a sentence"]))
conv_1d =layers.Conv1D(filters=32,
                       kernel_size=5,
                       activation="relu",
                       padding="valid")

conv_1d_output=conv_1d(embedding_test)

max_pool=layers.GlobalMaxPool1D()
max_pool_output=max_pool(conv_1d_output)


embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [101]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , 0.        , 0.03399948,
         0.        , 0.        , 0.04769304, 0.03062139, 0.        ,
         0.        , 0.015777  , 0.01623755, 0.        , 0.        ,
         0.        , 0.05472258, 0.00525452, 0.        , 0.051944  ,
         0.00567329, 0.        , 0.        , 0.02977181, 0.        ,
         0.00648806, 0.01499556, 0.02951892, 0.04299356, 0.01560639,
         0.        , 0.05294724],
        [0.00738041, 0.        , 0.03836391, 0.05402639, 0.0041481 ,
         0.01186075, 0.        , 0.        , 0.        , 0.00621176,
         0.        , 0.04446708, 0.00911371, 0.        , 0.00436583,
         0.01299233, 0.        , 0.0096913 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.01664212, 0.        ,
         0.        , 0.03983915, 0.07439598, 0.        , 0.02717502,
         0.        , 0.02985293],
        [0.04736824, 0.        , 0.02693348, 0.0136